# Elbow-Methode mit dem Wine-Datensatz

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_19/wine_elbow_method.ipynb)

In diesem Notebook lernen wir die **Elbow-Methode** kennen – das wichtigste Verfahren zur Bestimmung der optimalen Anzahl Cluster bei k-Means.

**Ziel:** Herausfinden, wie viele Cluster im Wine-Datensatz optimal sind, ohne die echten Labels zu kennen.

**Das Szenario:** Stellt euch vor, ihr seid Weinhändler und bekommt 178 Weine ohne Etiketten. Wie viele verschiedene Produzenten/Stile gibt es wohl?

## 1. Bibliotheken importieren und Daten laden

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_wine
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, adjusted_rand_score

# Für schönere Plots
plt.style.use('default')
sns.set_palette("husl")

print("🍷 Wine-Datensatz Elbow-Methode Notebook geladen!")

## 2. Wine-Datensatz verstehen

In [ ]:
# Wine-Datensatz aus sklearn laden
wine = load_wine()

print("📊 Wine-Datensatz Überblick:")
print(f"Anzahl Weine: {wine.data.shape[0]}")
print(f"Anzahl chemische Eigenschaften: {wine.data.shape[1]}")
print(f"Echte Produzenten: {wine.target_names}")

# Echte Verteilung (die wir "vergessen" für k-Means)
unique, counts = np.unique(wine.target, return_counts=True)
print("\n🏭 Echte Produzenten-Verteilung:")
for i, (name, count) in enumerate(zip(wine.target_names, counts)):
    print(f"  {name}: {count} Weine")

print("\n🔬 Chemische Eigenschaften:")
for i, feature in enumerate(wine.feature_names[:5]):
    print(f"  {i+1}. {feature}")
print("  ... und 8 weitere")

## 3. Daten vorbereiten (Skalierung!)

**Wichtig:** Der Wine-Datensatz hat sehr unterschiedliche Skalen (Alkohol 11-15% vs. Proline 278-1680 mg/L). Ohne Skalierung würde k-Means nur die großen Zahlen sehen!

In [ ]:
# Skalierungsproblem zeigen
print("⚠️ Skalierungsproblem demonstrieren:")
print(f"Alkohol: {wine.data[:, 0].min():.1f} - {wine.data[:, 0].max():.1f} %")
print(f"Proline: {wine.data[:, 12].min():.0f} - {wine.data[:, 12].max():.0f} mg/L")
print(f"Faktor-Unterschied: {wine.data[:, 12].max() / wine.data[:, 0].max():.0f}x!")

# Alle Features skalieren
scaler = StandardScaler()
X_scaled = scaler.fit_transform(wine.data)

print("\n✅ Nach StandardScaler:")
print(f"Mittelwerte (alle ~0): {X_scaled.mean(axis=0)[:3]}")
print(f"Std-Abweichungen (alle ~1): {X_scaled.std(axis=0)[:3]}")
print("→ Jetzt sind alle Features gleichberechtigt!")

## 4. Elbow-Methode durchführen

Wir probieren verschiedene k-Werte aus (k=1 bis k=10) und schauen, wie sich die **Intra-Cluster-Varianz** (Inertia) entwickelt.

In [ ]:
# Verschiedene k-Werte testen
k_range = range(1, 11)
inertias = []
silhouette_scores = []

print("🔍 Elbow-Methode läuft...")
for k in k_range:
    # k-Means für aktuelles k
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    
    # Inertia speichern
    inertias.append(kmeans.inertia_)
    
    # Silhouette Score (nur für k >= 2)
    if k >= 2:
        sil_score = silhouette_score(X_scaled, cluster_labels)
        silhouette_scores.append(sil_score)
    else:
        silhouette_scores.append(0)  # k=1 hat keinen Silhouette Score
    
    print(f"k={k}: Inertia={kmeans.inertia_:.2f}, Silhouette={silhouette_scores[-1]:.3f}")

print("✅ Elbow-Methode abgeschlossen!")

## 5. Elbow-Plot visualisieren

Jetzt schauen wir, wo die Kurve "abknickt" – das ist unser optimales k!

In [ ]:
# Elbow-Plot erstellen
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Inertia (Elbow-Methode)
ax1.plot(k_range, inertias, 'bo-', markersize=8, linewidth=2, label='Intra-Cluster-Varianz')
ax1.axvline(x=3, color='red', linestyle='--', alpha=0.7, label='k=3 (echte Anzahl)')
ax1.set_xlabel('Anzahl Cluster (k)')
ax1.set_ylabel('Intra-Cluster-Varianz (Inertia)')
ax1.set_title('Elbow-Methode: Wo knickt die Kurve ab?')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Annotationen für wichtige Punkte
ax1.annotate('Starker Abfall', xy=(2, inertias[1]), xytext=(2.5, inertias[1] + 50),
            arrowprops=dict(arrowstyle='->', color='orange'), fontsize=10)
ax1.annotate('Ellenbogen?\nk=3', xy=(3, inertias[2]), xytext=(4, inertias[2] + 30),
            arrowprops=dict(arrowstyle='->', color='red'), fontsize=10)

# Plot 2: Silhouette Score
k_range_sil = range(2, 11)  # Silhouette nur für k >= 2
ax2.plot(k_range_sil, silhouette_scores[1:], 'go-', markersize=8, linewidth=2, label='Silhouette Score')
ax2.axvline(x=3, color='red', linestyle='--', alpha=0.7, label='k=3 (echte Anzahl)')
ax2.set_xlabel('Anzahl Cluster (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Score: Höher = bessere Trennung')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Besten Silhouette Score markieren
best_k_sil = k_range_sil[np.argmax(silhouette_scores[1:])]
ax2.annotate(f'Maximum\nk={best_k_sil}', 
            xy=(best_k_sil, max(silhouette_scores[1:])), 
            xytext=(best_k_sil + 1, max(silhouette_scores[1:]) + 0.02),
            arrowprops=dict(arrowstyle='->', color='green'), fontsize=10)

plt.tight_layout()
plt.show()

print("👁️ Interpretation des Elbow-Plots:")
print("• Links: Suche den 'Ellenbogen' - wo die Kurve stark abflacht")
print("• Rechts: Silhouette Score als zusätzliche Bestätigung")
print(f"• Elbow-Methode deutet auf k={3} hin")
print(f"• Silhouette Score ist maximal bei k={best_k_sil}")

## 6. Detailanalyse: k=2, k=3, k=4 vergleichen

Schauen wir uns die "Kandidaten" genauer an:

In [ ]:
# Detailvergleich für k=2, 3, 4
candidate_ks = [2, 3, 4]
results = {}

print("🔬 Detailanalyse der Kandidaten:")
print(f"{'k':<3} {'Inertia':<10} {'Silhouette':<12} {'ARI':<8} {'Interpretation':<15}")
print("-" * 60)

for k in candidate_ks:
    # k-Means anwenden
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    
    # Metriken berechnen
    inertia = kmeans.inertia_
    silhouette = silhouette_score(X_scaled, labels)
    ari = adjusted_rand_score(wine.target, labels)  # Vergleich mit echten Labels
    
    # Cluster-Größen
    unique_labels, counts = np.unique(labels, return_counts=True)
    
    results[k] = {
        'inertia': inertia,
        'silhouette': silhouette,
        'ari': ari,
        'cluster_sizes': counts
    }
    
    # Qualitätsbewertung
    if ari > 0.8:
        quality = "Exzellent"
    elif ari > 0.6:
        quality = "Sehr gut"
    elif ari > 0.4:
        quality = "Gut"
    else:
        quality = "Mäßig"
    
    print(f"{k:<3} {inertia:<10.2f} {silhouette:<12.3f} {ari:<8.3f} {quality:<15}")
    print(f"    Cluster-Größen: {counts}")

print("\n💡 Erkenntnisse:")
print("• k=2: Einfache Teilung, aber übersieht Feinstrukturen")
print("• k=3: Bester Kompromiss - findet echte Produzenten gut (ARI={results[3]['ari']:.3f})")
print("• k=4: Überfitting - teilt echte Cluster unnötig auf")

## 7. Finale Entscheidung: k=3 anwenden

Basierend auf Elbow-Methode und Silhouette Score wählen wir k=3:

In [ ]:
# Finales k-Means mit k=3
print("🏆 Finales k-Means Clustering mit k=3:")
kmeans_final = KMeans(n_clusters=3, random_state=42, n_init=10)
final_labels = kmeans_final.fit_predict(X_scaled)
final_centers = kmeans_final.cluster_centers_

# Ergebnisse analysieren
final_ari = adjusted_rand_score(wine.target, final_labels)
final_silhouette = silhouette_score(X_scaled, final_labels)

print("📊 Finale Bewertung:")
print("• Adjusted Rand Index: {final_ari:.3f} (1.0 = perfekt)")
print("• Silhouette Score: {final_silhouette:.3f} (1.0 = perfekt getrennt)")

# Cluster-Größen vs. echte Verteilung
print("\n📈 Cluster-Verteilung:")
unique_final, counts_final = np.unique(final_labels, return_counts=True)
print("k-Means gefunden:")
for cluster, count in zip(unique_final, counts_final):
    print(f"  Cluster {cluster}: {count} Weine")

print("\nEchte Produzenten:")
unique_real, counts_real = np.unique(wine.target, return_counts=True)
for i, (name, count) in enumerate(zip(wine.target_names, counts_real)):
    print(f"  {name}: {count} Weine")

print("\n🎯 Fazit:")
if final_ari > 0.8:
    print("✅ Exzellent! k-Means hat die echten Produzenten sehr gut gefunden.")
elif final_ari > 0.6:
    print("✅ Sehr gut! k-Means findet die Hauptstrukturen.")
else:
    print("⚠️ Mäßig. Die Daten sind komplexer als erwartet.")

print("Die Elbow-Methode hat uns erfolgreich zu k=3 geführt! 🚀")

## 8. Bonus: Confusion Matrix - Wer wurde wie zugeordnet?

Schauen wir genauer, welche Weine k-Means welchem Cluster zugeordnet hat:

In [ ]:
# Confusion Matrix zwischen k-Means und echten Labels
from sklearn.metrics import confusion_matrix

# Einfache Zuordnung: Für jeden k-Means Cluster das häufigste echte Label finden
print("🔍 k-Means Cluster zu echten Labels zuordnen...")
cluster_to_label = {}

for cluster in range(3):
    # Alle Weine in diesem k-Means Cluster
    cluster_mask = final_labels == cluster
    cluster_true_labels = wine.target[cluster_mask]
    
    # Häufigstes echtes Label in diesem Cluster
    unique_labels, counts = np.unique(cluster_true_labels, return_counts=True)
    most_common_label = unique_labels[np.argmax(counts)]
    
    cluster_to_label[cluster] = most_common_label
    print(f"  k-Means Cluster {cluster} → {wine.target_names[most_common_label]} ({max(counts)}/{len(cluster_true_labels)} Weine)")

# Labels entsprechend zuordnen
final_labels_mapped = np.array([cluster_to_label[label] for label in final_labels])

# Confusion Matrix mit zugeordneten Labels
cm = confusion_matrix(wine.target, final_labels_mapped)

# Visualisierung
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=wine.target_names,
           yticklabels=wine.target_names)
plt.xlabel('k-Means Vorhersage (zugeordnet)')
plt.ylabel('Echte Produzenten')
plt.title('Confusion Matrix: k-Means vs. Echte Labels (zugeordnet)')
plt.show()

print("🧐 Confusion Matrix Interpretation:")
print("• Zeilen: Echte Produzenten")
print("• Spalten: k-Means Cluster") 
print("• Diagonale: Korrekt zugeordnete Weine")
print("• Off-Diagonale: 'Verwechslungen'")

# Genauigkeit pro Produzent
print("\n📊 Zuordnungsqualität pro Produzent:")
for i, producer in enumerate(wine.target_names):
    correct = cm[i, i] if i < len(cm) else 0
    total = sum(cm[i, :]) if i < len(cm) else 0
    accuracy = correct / total if total > 0 else 0
    print(f"  {producer}: {correct}/{total} = {accuracy:.2%} korrekt")

## 9. Zusammenfassung: Was haben wir gelernt?

In [ ]:
print("🎓 Wichtige Erkenntnisse aus der Elbow-Methode:")
print()
print("1️⃣ Elbow-Methode Vorgehen:")
print("   • Verschiedene k-Werte testen (k=1,2,3,...)")
print("   • Intra-Cluster-Varianz plotten")
print("   • 'Ellenbogen' suchen - wo Verbesserung stark abnimmt")
print()
print("2️⃣ Zusätzliche Metriken nutzen:")
print("   • Silhouette Score für Cluster-Qualität")
print("   • Bei bekannten Labels: ARI für Vergleich")
print("   • Cluster-Größen sollten sinnvoll sein")
print()
print("3️⃣ Praktische Tipps:")
print("   • Immer zuerst skalieren! (StandardScaler)")
print("   • Mehrere random_state testen")
print("   • Domain-Wissen einbeziehen")
print("   • Visualisierung hilft beim Verstehen")
print()
print("4️⃣ Unser Ergebnis:")
print("   • Elbow-Methode → k=3 optimal")
print("   • ARI = {final_ari:.3f} (sehr gute Übereinstimmung)")
print("   • k-Means fand die 3 Produzenten ohne Labels!")
print()
print("🚀 Die Elbow-Methode ist ein mächtiges Werkzeug zur")
print("   automatischen Bestimmung der optimalen Cluster-Anzahl!")